# Exploring Toronto
## Part 1 - Scraping Wiki and Creating the datatframe

In [1]:
#install beautifulsoup4, parser, request, uncomment and run if needed
#!pip install beautifulsoup4
#!pip install lxml
#!pip install requests

#import libs
import pandas as pd
import requests
from bs4 import BeautifulSoup
print('Libraries imported.')

Libraries imported.


In [2]:
Page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(Page.content,'lxml')
table = soup.find_all('table')[0] 
table_rows = table.find_all('tr')

#create a dictionary of all the rows
data = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        data.append(row)

#create dataframe with data dictionary
df = pd.DataFrame(data, columns=["Postal Code", "Borough", "Neighborhood"])

### The initial dataframe
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [ ]:
df.head()

### Removing 'Not assigned' Boroughs
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
# recreate dataframe where Borough not equals 'Not assigned'
df=df[df.Borough != 'Not assigned']
df.head(8)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue


### Replacing 'Not assigned' Neighborhood with Borough
If a cell has a borough but a Not assigned Neighborhoods, then the Neighborhoods will be the same as the borough. 

In [4]:
# Replace Neighborhood with Borough where Neighborhood = 'Not assigned'
for index, row in df.iterrows():
    if row['Neighborhood']=='Not assigned':
        row['Neighborhood']= row['Borough']
df.head(8)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue


### Combine Neighborhoods with Common Postal Codes
More than one Neighborhoods can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two Neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the Neighborhoods separated with a comma.

In [5]:
#set postal code and Borough as the index
df.set_index(['Postal Code','Borough'],inplace=True)

#Combine Rows of Multiindex DataFrame into Comma Separated Lists
df = df.groupby(level=['Postal Code','Borough'], sort=False).agg( ','.join)

In [6]:
#reset the index
df1=df.reset_index() 
df1.head(8)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North


### Final Dataframe Shape

In [7]:
df1.shape

(103, 3)

## Part 2 - Retrieving Geographical Coordinates

In [8]:
#import Geospatial_Coordinates.csv into a dataframe
df_pc= pd.read_csv('Documents/Coursera - IBM data science/capstone/Geospatial_Coordinates.csv')
df_pc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
#set postal code as the index
df1.set_index(['Postal Code'],inplace=True)
df_pc.set_index (['Postal Code'],inplace=True)

#combined df and df_pc based on PostalCode index
df2 = pd.concat([df1, df_pc], axis=1, join='inner')

#reset index
df3=df2.reset_index() 
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## Part 3 - Explore and cluster the Neighborhoods in Toronto.

In [10]:
#import libs

import numpy as np # library to handle data in a vecNYKized manner

# uncomment this line if you haven't completed the Foursquare API lab
#!pip install geopy  
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# uncomment this line if you haven't completed the Foursquare API lab
#!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Borough and Neighborhoods

In [11]:
print('The dataframe has {} boroughs and {} Neighborhoods.'.format(
        len(df3['Borough'].unique()),
        df3.shape[0]
    )
)

The dataframe has 11 boroughs and 103 Neighborhoods.


In [12]:
df3['Borough'].unique()


array(['North York', 'Downtown Toronto', "Queen's Park", 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

#### Visualize Toronto

In [48]:
address = 'Toronto, On'

geolocaNYK = Nominatim(user_agent="ny_explorer")
location_TOR = geolocaNYK.geocode(address)
latitude_TOR = location.latitude
longitude_TOR = location.longitude

#if geocode times out:
#The geograpical coordinate of Toronto are 43.653963, -79.387207.
latitude_TOR = 43.653963
longitude_TOR = -79.387207
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_TOR, longitude_TOR))



The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [49]:
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude_TOR, longitude_TOR], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(Neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

#### Visualize North York

In [11]:
NYK_data = df3[df3['Borough'].str.contains ("North York")].reset_index(drop=True)
NYK_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
3,M3B,North York,Don Mills North,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [13]:
address = 'North York, On'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

#if times out:
#The geograpical coordinate of North York are 43.7708175, -79.4132998.
#latitude = 43.7708175
#longitude = -79.4132998

print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of North York are 43.7708175, -79.4132998.


In [14]:
# create map of North York (NYK) using latitude and longitude values
map_NYK = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NYK_data['Latitude'], NYK_data['Longitude'], NYK_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NYK)  
    
map_NYK

In [50]:
# @hidden_cell
# 
CLIENT_ID = 'REMOVED' # your Foursquare ID
CLIENT_SECRET = 'REMOVED' # your Foursquare Secret
VERSION = '20190710'
#

### Explore Neighborhoods in North York

In [21]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### North York Venues

In [22]:
NYK_venues = getNearbyVenues(names=NYK_data['Neighborhood'],
                                   latitudes=NYK_data['Latitude'],
                                   longitudes=NYK_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Heights,Lawrence Manor
Don Mills North
Glencairn
Flemingdon Park,Don Mills South
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Fairview,Henry Farm,Oriole
Northwood Park,York University
Bayview Village
CFB Toronto,Downsview East
Silver Hills,York Mills
Downsview West
Downsview,North Park,Upwood Park
Humber Summit
Newtonbrook,Willowdale
Downsview Central
Bedford Park,Lawrence Manor East
Emery,Humberlea
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


#### The size of the resulting dataframe

In [23]:
print(NYK_venues.shape)
NYK_venues.head()

(241, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


#### The number of venues returned for each neighborhood

In [24]:
NYK_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor,Downsview North,Wilson Heights",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",22,22,22,22,22,22
"CFB Toronto,Downsview East",2,2,2,2,2,2
Don Mills North,6,6,6,6,6,6
Downsview Central,4,4,4,4,4,4
Downsview Northwest,4,4,4,4,4,4
Downsview West,6,6,6,6,6,6
"Downsview,North Park,Upwood Park",3,3,3,3,3,3


#### Unique categories can be curated from all the returned venues

In [26]:
print('There are {} uniques categories.'.format(len(NYK_venues['Venue Category'].unique())))

There are 105 uniques categories.


#### Analyze Each Neighborhood

In [27]:
# one hot encoding
NYK_onehot = pd.get_dummies(NYK_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NYK_onehot['Neighborhood'] = NYK_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NYK_onehot.columns[-1]] + list(NYK_onehot.columns[:-1])
NYK_onehot = NYK_onehot[fixed_columns]

NYK_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
NYK_onehot.shape

(241, 106)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
NYK_grouped = NYK_onehot.groupby('Neighborhood').mean().reset_index()
NYK_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Women's Store
0,"Bathurst Manor,Downsview North,Wilson Heights",0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.055556,0.0,...,0.055556,0.0,0.000000,0.0,0.0,0.0,0.055556,0.0,0.0,0.0
1,Bayview Village,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.250000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,"Bedford Park,Lawrence Manor East",0.0,0.0,0.045455,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.045455,0.0,0.045455,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,"CFB Toronto,Downsview East",0.0,0.5,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Don Mills North,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


#### The new size

In [31]:
NYK_grouped.shape

(23, 106)

#### Print each neighborhood along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in NYK_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NYK_grouped[NYK_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Downsview North,Wilson Heights----
                  venue  freq
0           Coffee Shop  0.11
1   Fried Chicken Joint  0.06
2        Sandwich Place  0.06
3         Shopping Mall  0.06
4  Fast Food Restaurant  0.06


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                venue  freq
0         Coffee Shop  0.09
1  Italian Restaurant  0.09
2      Sandwich Place  0.05
3    Greek Restaurant  0.05
4   Indian Restaurant  0.05


----CFB Toronto,Downsview East----
                venue  freq
0             Airport   0.5
1                Park   0.5
2            Pharmacy   0.0
3       Movie Theater   0.0
4  Miscellaneous Shop   0.0


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center  0.17
1  Caribbean Restaurant  0.17
2                  Café  0.17
3   

#### Put above data into a dataframe

In [33]:
#function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
#Create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicaNYKs = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicaNYKs[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NYK_grouped['Neighborhood']

for ind in np.arange(NYK_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NYK_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Sandwich Place,Bank,Restaurant,Diner,Pizza Place,Shopping Mall,Pharmacy,Deli / Bodega,Supermarket
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,"Bedford Park,Lawrence Manor East",Coffee Shop,Italian Restaurant,Butcher,Sandwich Place,Greek Restaurant,Grocery Store,Indian Restaurant,Comfort Food Restaurant,Japanese Restaurant,Juice Bar
3,"CFB Toronto,Downsview East",Park,Airport,Dog Run,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
4,Don Mills North,Caribbean Restaurant,Café,Gym / Fitness Center,Baseball Field,Basketball Court,Japanese Restaurant,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store


### Cluster Neighborhoods

In [39]:
# set number of clusters
kclusters = 5

NYK_grouped_clustering = NYK_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NYK_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 1, 0, 3, 0, 0, 1, 3])

In [40]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

NYK_merged = NYK_data

# merge NYK_data with NYK_data to add latitude/longitude for each neighborhood
NYK_merged1 = pd.merge(NYK_merged, neighborhoods_venues_sorted, on='Neighborhood', how='inner')

NYK_merged1.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Fast Food Restaurant,Bus Stop,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,0,Intersection,French Restaurant,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Diner,Clothing Store,Comfort Food Restaurant,Construction & Landscaping
2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,0,Furniture / Home Store,Clothing Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Gift Shop,Accessories Store,Vietnamese Restaurant
3,M3B,North York,Don Mills North,43.745906,-79.352188,0,Caribbean Restaurant,Café,Gym / Fitness Center,Baseball Field,Basketball Court,Japanese Restaurant,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
4,M6B,North York,Glencairn,43.709577,-79.445073,0,Asian Restaurant,Pub,Japanese Restaurant,Park,Women's Store,Diner,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping


### Visualize the resulting clusters

In [41]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NYK_merged1['Latitude'], NYK_merged1['Longitude'], NYK_merged1['Neighborhood'], NYK_merged1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[(cluster)-1],
        fill=True,
        fill_color=rainbow[(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters